In [1]:
import pandas as pd
import numpy as np
from Classes import Sortino
from Classes import PortfolioClassifier
from sklearn.cluster import KMeans
import pickle

In [2]:
prices_df = pd.read_excel("daily_dbs/daily_dbs_sp50&nasdaq.xlsx")
market_features_df = pd.read_excel("daily_dbs/daily_market_features.xlsx")
rfr_df = pd.read_csv('daily_dbs/risk_free_rate.csv')

primero vamos a llenar los NaNs que tiene el archivo de risk free rate

In [3]:
rfr_df.columns = [col.strip() for col in rfr_df.columns]
rfr_df.rename(columns={"observation_date": "Date", "DGS10": "rfr"}, inplace=True)
rfr_df['Date'] = pd.to_datetime(rfr_df['Date'])

rfr_df = rfr_df.sort_values("Date")
rfr_df['rfr'] = rfr_df['rfr'].fillna(method='ffill')

rfr_df.to_csv('daily_dbs/risk_free_rate.csv', index=False)

C:\Users\renat\AppData\Local\Temp\ipykernel_31728\121179686.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rfr_df['rfr'] = rfr_df['rfr'].fillna(method='ffill')


Vamos a generar el dataset de portafolios con pesos aleatorios y vamos a calcular su rendimiento

In [4]:
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
prices_df.set_index('Date', inplace=True)
returns_df = np.log(prices_df/prices_df.shift(1)).dropna()
returns_df.to_csv('daily_dbs/returns_df.csv')

selección aleatoria de activos

In [5]:
returns_df = pd.read_csv("daily_dbs/returns_df.csv")

np.random.seed(42)
n = 20
tickers = returns_df.columns.tolist()
selected_assets = np.random.choice(tickers, n, replace=False).tolist()
selected_assets

['LIN',
 'KIM',
 'TT',
 'AME',
 'IT',
 'MO',
 'DTE',
 'DOV',
 'EXPE',
 'PG',
 'V',
 'COST',
 'KLAC',
 'ZBH',
 'KO',
 'GILD',
 'WSM',
 'ALL',
 'EQIX',
 'INTC']

In [6]:
# 1. Crear el clasificador
clf = PortfolioClassifier("market_caps.xlsx")

ref1 = clf.get_random_portfolio(n=20)
ref2 = clf.get_random_portfolio(n=20)
clf.add_reference_portfolio(ref1)  
clf.add_reference_portfolio(ref2)

# GUARDAR EL CLASIFICADOR AQUÍ (después de agregar las referencias)
with open('portfolio_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

x_temp = clf.get_random_portfolio(n=20)
print("Portafolio temporal:", x_temp)


etiqueta = clf.classify(x_temp)
print("Etiqueta asignada:", etiqueta)

Portafolio temporal: ['TRMB', 'HAS', 'CMI', 'CDNS', 'USB', 'WM', 'CMS', 'CNP', 'DECK', 'SBAC', 'PEG', 'CI', 'AXP', 'YUM', 'MTCH', 'EXPD', 'WMT', 'WST', 'NUE', 'EQIX']
Etiqueta asignada: 1


In [7]:
# Traslado hacia los 3 svr's (high, mid o low cap)



llamamos a la clase de sortino que hace toda la chamba :V

In [8]:
rfr_path = 'daily_dbs/risk_free_rate.csv'
sortino = Sortino(returns_df, rfr_path, selected_assets)

sortino.generate_multiple_weights(num_combinations=10)
sortino.calculate_portfolio_returns()
sortino.calculate_sortino_ratio()

sortino_dataset = sortino.create_portfolio_dataset()
sortino_dataset.head()

Generadas 10 combinaciones de pesos para cada fecha.
Rendimientos del portafolio calculados para cada combinación de pesos.
Ratios de Sortino calculados para cada combinación de pesos.


,Date,Weight_LIN,Weight_KIM,Weight_TT,Weight_AME,Weight_IT,Weight_MO,Weight_DTE,Weight_DOV,Weight_EXPE,...,Weight_KLAC,Weight_ZBH,Weight_KO,Weight_GILD,Weight_WSM,Weight_ALL,Weight_EQIX,Weight_INTC,Portfolio_Returns,Sortino_Ratio
0,2010-01-05,7.762513,1.354741,2.652582,6.159358,9.199787,5.961198,2.531425,7.212902,7.850159,...,5.268331,4.133704,3.157799,1.060803,0.567459,10.169771,8.987876,3.765413,-0.001547,-0.187525
1,2010-01-05,8.034707,5.683155,4.051956,4.140169,0.699378,0.770083,5.058980,4.649705,1.786376,...,7.817450,8.181221,8.363303,0.469176,6.189235,4.584312,5.927199,8.134226,-0.001343,-0.164917
2,2010-01-05,5.899832,7.176935,7.433368,7.189942,3.653719,1.908585,3.401273,7.647984,1.257131,...,7.401382,7.549055,2.029567,7.783441,5.075425,3.003109,6.072628,4.130277,-0.001027,-0.130076
3,2010-01-05,3.761737,7.017009,2.475306,3.286677,4.323670,2.524648,4.032576,5.687307,7.374181,...,6.777713,2.363667,3.983023,4.754225,0.824935,5.258346,4.342420,7.982601,-0.002843,-0.330729
4,2010-01-05,9.238108,5.252512,3.048379,0.410002,8.734993,8.682774,2.389967,6.569464,0.712621,...,9.407498,6.583766,3.629508,6.963300,8.646296,9.056745,0.546633,3.726999,0.001265,0.123293


merge con los market features

In [9]:
sortino_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37730 entries, 0 to 37729
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               37730 non-null  datetime64[ns]
 1   Weight_LIN         37730 non-null  float64       
 2   Weight_KIM         37730 non-null  float64       
 3   Weight_TT          37730 non-null  float64       
 4   Weight_AME         37730 non-null  float64       
 5   Weight_IT          37730 non-null  float64       
 6   Weight_MO          37730 non-null  float64       
 7   Weight_DTE         37730 non-null  float64       
 8   Weight_DOV         37730 non-null  float64       
 9   Weight_EXPE        37730 non-null  float64       
 10  Weight_PG          37730 non-null  float64       
 11  Weight_V           37730 non-null  float64       
 12  Weight_COST        37730 non-null  float64       
 13  Weight_KLAC        37730 non-null  float64       
 14  Weight

In [10]:
database_daily = pd.merge(sortino_dataset, market_features_df, on="Date", how="inner")

In [11]:
database_daily.head()

,Date,Weight_LIN,Weight_KIM,Weight_TT,Weight_AME,Weight_IT,Weight_MO,Weight_DTE,Weight_DOV,Weight_EXPE,...,Sortino_Ratio,Treasury Bond 10Y,Treasury Bond 3M,WTI Index,Yield Curve Spread (10Y - 2Y),USD Index,MOM,VVIX,SMB,HML
0,2010-01-05,7.762513,1.354741,2.652582,6.159358,9.199787,5.961198,2.531425,7.212902,7.850159,...,-0.187525,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
1,2010-01-05,8.034707,5.683155,4.051956,4.140169,0.699378,0.770083,5.058980,4.649705,1.786376,...,-0.164917,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
2,2010-01-05,5.899832,7.176935,7.433368,7.189942,3.653719,1.908585,3.401273,7.647984,1.257131,...,-0.130076,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
3,2010-01-05,3.761737,7.017009,2.475306,3.286677,4.323670,2.524648,4.032576,5.687307,7.374181,...,-0.330729,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
4,2010-01-05,9.238108,5.252512,3.048379,0.410002,8.734993,8.682774,2.389967,6.569464,0.712621,...,0.123293,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05


In [12]:
database_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37700 entries, 0 to 37699
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Date                           37700 non-null  datetime64[ns]
 1   Weight_LIN                     37700 non-null  float64       
 2   Weight_KIM                     37700 non-null  float64       
 3   Weight_TT                      37700 non-null  float64       
 4   Weight_AME                     37700 non-null  float64       
 5   Weight_IT                      37700 non-null  float64       
 6   Weight_MO                      37700 non-null  float64       
 7   Weight_DTE                     37700 non-null  float64       
 8   Weight_DOV                     37700 non-null  float64       
 9   Weight_EXPE                    37700 non-null  float64       
 10  Weight_PG                      37700 non-null  float64       
 11  Weight_V       

In [3]:
database_bt = pd.merge(prices_df, market_features_df, on="Date", how="inner")
database_bt.head()

,Date,CTAS,RTX,WEC,MAA,AES,FAST,ANSS,ED,EQIX,...,COO,Treasury Bond 10Y,Treasury Bond 3M,WTI Index,Yield Curve Spread (10Y - 2Y),USD Index,MOM,VVIX,SMB,HML
0,2010-01-04,5.795176,31.163885,14.957690,27.001556,9.061471,3.075539,44.639999,24.718466,82.410835,...,9.377275,3.85,0.08,81.52,2.76,77.53,0.58,20.04,-0.14,0.21
1,2010-01-05,5.786392,30.698355,14.762187,26.979151,8.968672,3.139202,43.639999,24.397079,81.643608,...,9.486571,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
2,2010-01-06,5.801763,30.537390,14.912576,27.068806,8.875872,3.346286,43.349998,24.658539,82.388283,...,9.481605,3.85,0.06,83.12,2.84,77.49,-0.05,19.16,-0.33,0.83
3,2010-01-07,5.852271,30.667902,14.882504,27.197664,8.882501,3.403363,43.459999,24.571383,80.703362,...,9.580969,3.85,0.05,82.60,2.82,77.91,-0.87,19.06,-0.10,0.09
4,2010-01-08,5.909366,30.728806,14.969725,27.018373,9.134392,3.437755,43.759998,24.647646,80.312218,...,9.426954,3.83,0.05,82.74,2.87,77.47,0.20,18.13,0.18,-0.28


In [4]:
database_bt.to_csv("daily_dbs/dbs_backtesting.csv")

In [5]:
database_bt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3771 entries, 0 to 3770
Columns: 435 entries, Date to HML
dtypes: datetime64[ns](1), float64(434)
memory usage: 12.5 MB
